In [0]:
from google.colab import drive
drive.mount('/content/drive')
root_path = 'drive/My Drive/Final_year_project/Final_Project_Notes/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp "/content/drive/My Drive/Final_year_project/Final_Project_Notes/Python_Code/resnet_build.py" "resnet_build.py"

In [0]:
from keras.applications import ResNet50,InceptionV3
from keras.models import Model
from keras.layers import Dropout
from keras import backend as K
from resnet_build import ResnetBuilder
from keras.layers import Input,Activation,Flatten,Conv2D,MaxPooling2D,ZeroPadding2D,AveragePooling2D,GlobalAveragePooling2D,GlobalMaxPooling2D,BatchNormalization
from keras.layers import Dense
from keras.engine.topology import get_source_inputs
from keras.utils.data_utils import get_file
from keras.applications.resnet50 import resnet50
identity_block, conv_block = resnet50.identity_block, resnet50.conv_block

In [0]:
def resnet_trained(n_retrain_layers = 0):
    K.set_image_data_format('channels_last')
    base_model = ResNet50(include_top=False,input_shape=(224,224,3))
    features = GlobalAveragePooling2D()(base_model.output)
    model = Model(inputs=base_model.input, outputs=features)
    model = _set_n_retrain(model,n_retrain_layers)
    return model

In [0]:
def inception(n_retrain_layers = 0):
    K.set_image_data_format('channels_last')
    base_model = InceptionV3(include_top=False, input_shape=(224, 224, 3))
    features = GlobalAveragePooling2D()(base_model.output)
    model = Model(inputs=base_model.input, outputs=features)
    model = _set_n_retrain(model,n_retrain_layers)
    return model

In [0]:
def resnet_trained_2(n_retrain_layers = 0):
    K.set_image_data_format('channels_last')
    base_model = ResNet50(include_top=False, input_shape=(224, 224, 3))
    features = GlobalAveragePooling2D()(base_model.output)
    model = Model(inputs=base_model.input, outputs=features)
    model = _set_n_retrain(model, n_retrain_layers,reinit=True)
    return model

In [0]:
def empty_resnet():
    K.set_image_data_format('channels_last')
    base_model = ResNet50(weights=None,include_top=False,input_shape=(224,224,3))
    features = GlobalAveragePooling2D()(base_model.output)
    model = Model(inputs=base_model.input, outputs=features)
    return model

In [0]:
def resnet18():
    return ResnetBuilder.build_resnet_18((3,224,224),25)
def resnet34():
    return ResnetBuilder.build_resnet_34((3,224,224),25)
def resnet101():
    return ResnetBuilder.build_resnet_101((3,224,224),25)
def resnet152():
    return ResnetBuilder.build_resnet_152((3,224,224),25)

In [0]:
def custom_resnet(n=0,dp_rate=0):


    WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels.h5'
    WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

    # Determine proper input shape
    #input_shape = _obtain_input_shape(input_shape,default_size=224,min_size=197,data_format=K.image_data_format(),include_top=include_top)

    img_input = Input(shape=(224,224,3))

    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    x = ZeroPadding2D((3, 3))(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')


    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = Dropout(dp_rate)(x)

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = Dropout(dp_rate)(x)

    x = AveragePooling2D((7, 7), name='avg_pool')(x)

    x = Flatten()(x)
    x = Dense(25, activation='softmax', name='fc1000')(x)
    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.

    inputs = img_input
    # Create model.
    model = Model(inputs, x, name='resnet50')

    # load weights

    weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                            WEIGHTS_PATH_NO_TOP,
                            cache_subdir='models',
                            md5_hash='a268eb855778b3df3c7506639542a6af')
    model.load_weights(weights_path,by_name=True)

    split_value = True # len(model.layers) + 1 - n
    for layer in model.layers[:split_value]:
        layer.trainable = False
    for layer in model.layers[split_value:]:
        layer.trainable = True

    return model

In [0]:
def resnet_dropout(include_top=False, weights='imagenet', input_tensor = None, pooling='avg', input_shape=(224,224,3),classes=25,dp_rate=0.,n_retrain_layers=0):



    WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels.h5'
    WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')

    # Determine proper input shape
    #input_shape = _obtain_input_shape(input_shape,default_size=224,min_size=197,data_format=K.image_data_format(),include_top=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    x = ZeroPadding2D((3, 3))(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')


    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = Dropout(dp_rate)(x)
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')


    x = AveragePooling2D((7, 7), name='avg_pool')(x)

    if include_top:
        x = Flatten()(x)
        x = Dense(classes, activation='softmax', name='fc1000')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='resnet50')

    # load weights
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models',
                                    md5_hash='a7b3fe01876f51b976af0dea6bc144eb')
        else:
            weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models',
                                    md5_hash='a268eb855778b3df3c7506639542a6af')
        model.load_weights(weights_path)

    split_value = len(model.layers) + 1 - n_retrain_layers
    for layer in model.layers[:split_value]:
        layer.trainable = False
    for layer in model.layers[split_value:]:
        layer.trainable = True

    return model

In [0]:
def _get_weighted_layers(model):
    res=[]
    for layer in model.layers:
        if len(layer.get_weights()) != 0:
            res.append(layer.name)
    return res

In [0]:
def _set_n_retrain(model,n,reinit=False):
    w_layers = _get_weighted_layers(model)
    if reinit:
        empty_model = empty_resnet()

    if n > len(w_layers):
        n == len(w_layers)
    if n>0:
        if reinit:
            for layer, layer_empty in zip(model.layers, empty_model.layers):
                if layer.name in w_layers[-n:]:
                    layer.trainable = True
                    w = layer_empty.get_weights()
                    layer.set_weights(w)
                else:
                    layer.trainable = False
        else :
            for layer in model.layers:
                if layer.name in w_layers[-n:]:
                    layer.trainable = True
                else:
                    layer.trainable = False

    else :
        for layer in model.layers:
            layer.trainable = False

    return model


In [0]:
if __name__ == '__main__':
    model = resnet_trained(1)
    print(model.summary())

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________

In [0]:
from keras import backend as K
import os
from os.path import join
import argparse


# PARSING ARGUMENTS

parser = argparse.ArgumentParser(description='Description')

parser.add_argument('-m', action="store", default='resnet',dest='model_name',help='Name of the model [alexnet_empty|decaf6|resnet|inception|inceptionv4|resnet2|empty_resnet|resnet_dropout|resnet_18|resnet_34|resnet_101|resnet_152|custom_resnet')
parser.add_argument('-b', action="store", default=32, type=int,dest='batch_size',help='Size of the batch.')
parser.add_argument('-e', action="store",default=10,type=int,dest='epochs',help='Number of epochs')
parser.add_argument('-f', action="store", default=False, type=bool,dest='horizontal_flip',help='Set horizontal flip or not [True|False]')
parser.add_argument('-n', action="store", default=0, type=int,dest='n_layers_trainable',help='Set the number of last trainable layers')
parser.add_argument('-d', action="store", default=0, type=float,dest='dropout_rate',help='Set the dropout_rate')

parser.add_argument('-p', action="store",dest='preprocessing',help='Set imagenet preprocessing or not')

parser.add_argument('--distortions', action="store", type=float,dest='disto',default=0.,help='Activate distortions or not')

TRAINING_PATH = '/content/drive/My Drive/Final_year_project/Final_Project_Notes/Paintings_Small/Train'
VAL_PATH = '/content/drive/My Drive/Final_year_project/Final_Project_Notes/Paintings_Small/Val'

args = parser.parse_args()

model_name = args.model_name
batch_size = args.batch_size
epochs = args.epochs
flip = args.horizontal_flip

n_layers_trainable = args.n_layers_trainable
dropout_rate = args.dropout_rate

params = vars(args)

# BUILDING MODEL


if model_name =='alexnet_empty':
    K.set_image_data_format('channels_first')
    size = (227, 227)
    model = alexnet(weights=None)
    for layer in model.layers:
        layer.trainable = True

elif model_name =='decaf6':
    K.set_image_data_format('channels_first')
    size = (227, 227)
    base_model = decaf()
    predictions = Dense(25, activation='softmax')(base_model.output)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False

elif model_name =='resnet':
    K.set_image_data_format('channels_last')
    size = (224,224)

    base_model = resnet_trained(n_layers_trainable)
    predictions = Dense(25, activation='softmax')(base_model.output)
    model = Model(inputs=base_model.input, outputs=predictions)

elif model_name =='inception':
    K.set_image_data_format('channels_last')
    size = (224,224)

    base_model = inception(n_layers_trainable)
    predictions = Dense(25, activation='softmax')(base_model.output)
    model = Model(inputs=base_model.input, outputs=predictions)

elif model_name =='inceptionv4':
    K.set_image_data_format('channels_last')
    size = (299,299)
    model = inception_v4()


elif model_name=='resnet2':
    K.set_image_data_format('channels_last')
    size=(224,224)
    base_model = resnet_trained_2(n_layers_trainable)
    predictions = Dense(25, activation='softmax')(base_model.output)
    model = Model(inputs=base_model.input, outputs=predictions)

elif model_name =='empty_resnet':
    K.set_image_data_format('channels_last')
    size = (224,224)

    base_model = empty_resnet()
    predictions = Dense(25, activation='softmax')(base_model.output)
    model = Model(inputs=base_model.input, outputs=predictions)

elif model_name=='resnet_dropout':
    K.set_image_data_format('channels_last')
    size = (224, 224)
    base_model = resnet_dropout(dp_rate=dropout_rate,n_retrain_layers=n_layers_trainable)
    predictions = Dense(25, activation='softmax')(base_model.output)
    model = Model(inputs=base_model.input, outputs=predictions)

elif model_name=='resnet_18':
    size = (224, 224)
    K.set_image_data_format('channels_last')
    model =  resnet18()

elif model_name=='resnet_34':
    size = (224, 224)
    K.set_image_data_format('channels_last')
    model =  resnet34()

elif model_name=='resnet_101':
    size = (224, 224)
    K.set_image_data_format('channels_last')
    model =  resnet101()

elif model_name=='resnet_152':
    size = (224, 224)
    K.set_image_data_format('channels_last')
    model =  resnet152()
elif model_name == 'custom_resnet':
    size = (224, 224)
    K.set_image_data_format('channels_last')
    model = custom_resnet(dp_rate=dropout_rate)
else:
    print("The model name doesn't exist")

model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
train_model_from_directory(TRAINING_PATH,model,model_name=model_name,target_size=size,validation_path=VAL_PATH,epochs = epochs,batch_size = batch_size,horizontal_flip=flip,params=params,preprocessing=args.preprocessing,distortions=args.disto)


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


NameError: ignored